In [7]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

In [8]:
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Extract data for ALICE and BOB channels
alice_CIRs = data_cir[:, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = data_cir[:, eve_channel, :, :]  # Shape: (9797, 251, 2)

# ALICE features - real, imaginary, magnitude
alice_real = alice_CIRs[:, :, 0]
alice_imag = alice_CIRs[:, :, 1]
alice_features = np.hstack((alice_real, alice_imag)) # (9797, 502)

# EVE features - real, imaginary, magnitude
eve_real = eve_CIRs[:, :, 0]
eve_imag = eve_CIRs[:, :, 1]
eve_features = np.hstack((eve_real, eve_imag)) # (9797, 502)

# Labels for ALICE and EVE.
alice_labels = np.zeros(alice_features.shape[0])  # Label '0' for Alice. (9797,)
eve_labels = np.ones(alice_features.shape[0])     # Label '1' for Eve. (9797,)

# Atoms
atoms = np.vstack((alice_features, eve_features))  # (19594, 753)
true_labels = np.hstack((alice_labels, eve_labels))  # (19594,)


In [9]:
# Step 3: Split the Data into Training and Test Sets
trainData, testData, trainLabel, testLabel = train_test_split(atoms, true_labels, test_size=0.2, random_state=42, stratify=true_labels)

# Step 4: Form the Dictionary D
D = trainData.T

# Dictionary shape (502, 15675)
# trainData (15675, 502) -- > 80% of 19594
# testData (3919, 502)   -- > 20% of 19594
# trainLabel (15675,) -- > 80% of 19594
# testLabel (3919,)  -- > 20% of 19594

# print(f'Dictionary shape {D.shape}')
# print(f'trainData shape {trainData.shape}')
# print(f'testData shape {testData.shape}')
# print(f'trainLabel shape {trainLabel.shape}')
# print(f'testLabel shape {testLabel.shape}')


Dictionary shape (502, 15675)
trainData shape (15675, 502)
testData shape (3919, 502)
trainLabel shape (15675,)
testLabel shape (3919,)


In [10]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=10):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# D - (502, 160)
# tSample - (502,)

In [11]:
# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    # print(f'Dictionary: {D.shape}')
    # print(f'Coeff class: {coef_class.shape}')
    # print(f'Reconstructed signal{reconstructed_signal.shape}')
    return residual

# tSample - (502,)
# coefficients - (160,)
# class_indices - (80,)
# reconstructed_signal - (502,)


In [12]:
# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # tSample - (502,)
    # D - (502, 160)
    # trainLabel - (160,)
    # coefficients - (160,)
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    # print(coefficients)
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class


In [13]:
# Step 8: Classifying Test Data and Evaluating the Model

# testData (40, 502)
predictions = []
for testSample in testData:
    predicted_class = classify_signal(testSample, D, trainLabel)
    predictions.append(predicted_class)


# Convert predictions to a numpy array for comparison
predictions = np.array(predictions)
# print(predictions)

# Step 9: Calculate Accuracy
accuracy = np.mean(predictions == testLabel)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

Classification Accuracy: 67.34%


In [15]:
# Calculate confusion matrix
print(f"\nTotal testing channel: {testData.shape}")

tn, fp, fn, tp = confusion_matrix(testLabel, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Total testing channel: (3919, 502)
tp: 1637
tn: 1002
fp: 958
fn: 322
MDR: 0.4887755102040816
FAR: 0.16436957631444615
AR: 0.6734274567407361
